# VESC Motor Controller - Advanced Control Examples

This notebook demonstrates advanced motor control techniques including current control, data logging, and sensor monitoring.

## Setup

In [ ]:
# Setup Python path to find student_api from parent directory
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

from student_api import VESCStudentAPI
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

vesc_api = VESCStudentAPI() # Create and start VESC API

if vesc_api.start():
    time.sleep(6)  # Wait for controller discovery
    
    connected_controllers = vesc_api.get_connected_controllers() # Check if VESC controller 74 is connected
    vesc = vesc_api.get_controller(74)
    
    if 74 in connected_controllers and vesc.is_connected():
        voltage = vesc.get_input_voltage()
        print(f"✅ Connected to VESC controller! Battery: {voltage:.1f}V")
    else:
        print("❌ VESC controller not found or not responding! Check that it is powered ON.")
else:
    print("❌ Failed to start VESC system")

## Current Control Mode

Demonstrates precise current control for torque management:

In [ ]:
# SAFETY CHECK
MOTOR_CONTROL_ENABLED = False  # Set to True to enable motor commands

if MOTOR_CONTROL_ENABLED:
    print("Testing current control...")
    
    # Set different current levels
    current_levels = [1.0, 2.0, 3.0, 2.0, 1.0, 0.0]  # Amperes
    
    for target_current in current_levels:
        print(f"Setting current to {target_current}A...")
        vesc.set_current(target_current)
        
        time.sleep(1)
        
        # Read actual values
        actual_current = vesc.get_motor_current()
        rpm = vesc.get_rpm()
        
        print(f"  Actual current: {actual_current}A, RPM: {rpm}")
    
    # Stop
    vesc.set_current(0)
    print("Current control test complete.")
else:
    print("Current control test skipped (safety mode).")

## Data Logging and Visualization

Collect telemetry data over time and visualize it:

In [ ]:
# Data logging
def log_vesc_data(duration_seconds=30, sample_rate=10):
    """Log VESC data for analysis"""
    
    timestamps = []
    rpm_data = []
    current_data = []
    voltage_data = []
    temp_data = []
    
    print(f"Logging data for {duration_seconds} seconds at {sample_rate}Hz...")
    
    start_time = time.time()
    sample_interval = 1.0 / sample_rate
    
    while time.time() - start_time < duration_seconds:
        current_time = time.time() - start_time
        
        # Collect data
        rpm = vesc.get_rpm() or 0
        current = vesc.get_motor_current() or 0
        voltage = vesc.get_input_voltage() or 0
        temp = vesc.get_motor_temperature() or 0
        
        timestamps.append(current_time)
        rpm_data.append(rpm)
        current_data.append(current)
        voltage_data.append(voltage)
        temp_data.append(temp)
        
        time.sleep(sample_interval)
    
    return {
        'time': timestamps,
        'rpm': rpm_data,
        'current': current_data,
        'voltage': voltage_data,
        'temperature': temp_data
    }

# Log data for 15 seconds
data = log_vesc_data(duration_seconds=15, sample_rate=5)
print(f"Collected {len(data['time'])} data points.")

In [ ]:
# Visualize the logged data
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('VESC Telemetry Data Over Time')

# RPM plot
axes[0,0].plot(data['time'], data['rpm'])
axes[0,0].set_title('Motor RPM')
axes[0,0].set_xlabel('Time (s)')
axes[0,0].set_ylabel('RPM')
axes[0,0].grid(True)

# Current plot
axes[0,1].plot(data['time'], data['current'])
axes[0,1].set_title('Motor Current')
axes[0,1].set_xlabel('Time (s)')
axes[0,1].set_ylabel('Current (A)')
axes[0,1].grid(True)

# Voltage plot
axes[1,0].plot(data['time'], data['voltage'])
axes[1,0].set_title('Input Voltage')
axes[1,0].set_xlabel('Time (s)')
axes[1,0].set_ylabel('Voltage (V)')
axes[1,0].grid(True)

# Temperature plot
axes[1,1].plot(data['time'], data['temperature'])
axes[1,1].set_title('Motor Temperature')
axes[1,1].set_xlabel('Time (s)')
axes[1,1].set_ylabel('Temperature (°C)')
axes[1,1].grid(True)

plt.tight_layout()
plt.show()

# Print statistics
print("\n=== Data Statistics ===")
print(f"Average RPM: {np.mean(data['rpm']):.1f}")
print(f"Average Current: {np.mean(data['current']):.2f}A")
print(f"Average Voltage: {np.mean(data['voltage']):.1f}V")
print(f"Max Temperature: {np.max(data['temperature']):.1f}°C")

In [ ]:
if MOTOR_CONTROL_ENABLED:
    print("Testing brake control...")
    
    # First, spin up the motor
    print("Spinning up motor...")
    vesc.set_duty_cycle(0.2)
    time.sleep(3)
    
    initial_rpm = vesc.get_rpm()
    print(f"Initial RPM: {initial_rpm}")
    
    # Apply regenerative braking
    print("Applying regenerative brake...")
    vesc.set_brake_current(2.0)  # 2A braking current
    
    # Monitor braking
    for i in range(10):
        rpm = vesc.get_rpm()
        current = vesc.get_motor_current()
        print(f"Braking - RPM: {rpm}, Current: {current}A")
        time.sleep(0.5)
        
        if abs(rpm or 0) < 100:  # Nearly stopped
            break
    
    # Stop braking
    vesc.set_brake_current(0)
    print("Braking complete.")
else:
    print("Brake control test skipped (safety mode).")

## Braking Control

Demonstrates regenerative braking control:

In [ ]:
# Read all sensor data
print("=== Complete Sensor Reading ===")

# Get all telemetry
telemetry = vesc.get_all_telemetry()

# Display organized data
print("\n--- Motor Status ---")
motor = telemetry['motor']
print(f"RPM: {motor['rpm']}")
print(f"Current: {motor['current']} A")
print(f"Duty Cycle: {motor['duty_cycle']}")

print("\n--- Power Status ---")
power = telemetry['power']
print(f"Input Voltage: {power['input_voltage']} V")
print(f"Input Current: {power['input_current']} A")
print(f"Energy Consumed: {power['amp_hours_consumed']} Ah")
print(f"Energy Recovered: {power['amp_hours_charged']} Ah")

print("\n--- Temperature Status ---")
temps = telemetry['temperatures']
print(f"FET Temperature: {temps['fet']}°C")
print(f"Motor Temperature: {temps['motor']}°C")

print("\n--- Sensor Inputs ---")
sensors = telemetry['sensors']
print(f"Tachometer: {sensors['tachometer']}")
print(f"PID Position: {sensors['pid_position']}")
print(f"ADC EXT: {sensors['adc_ext']} V")
print(f"ADC EXT2: {sensors['adc_ext2']} V")
print(f"ADC EXT3: {sensors['adc_ext3']} V")
print(f"Servo/PPM: {sensors['servo_value']}")

## Data Export

Save collected data for further analysis:

In [ ]:
import json
import csv

# Save telemetry snapshot as JSON
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename_json = f"vesc_telemetry_{timestamp}.json"

with open(filename_json, 'w') as f:
    json.dump(telemetry, f, indent=2)

print(f"Telemetry data saved to {filename_json}")

# Save logged data as CSV
if 'data' in locals():
    filename_csv = f"vesc_timeseries_{timestamp}.csv"
    
    with open(filename_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Time', 'RPM', 'Current', 'Voltage', 'Temperature'])
        
        for i in range(len(data['time'])):
            writer.writerow([
                data['time'][i],
                data['rpm'][i],
                data['current'][i],
                data['voltage'][i],
                data['temperature'][i]
            ])
    
    print(f"Time series data saved to {filename_csv}")

print("\nData export complete!")

## Final Safety Check

Always ensure the motor is stopped:

In [ ]:
# Ensure motor is completely stopped
vesc.set_duty_cycle(0)
vesc.set_current(0)
vesc.set_brake_current(0)

# Verify it's stopped
time.sleep(1)
final_rpm = vesc.get_rpm()
final_current = vesc.get_motor_current()

print(f"Final Status:")
print(f"  RPM: {final_rpm}")
print(f"  Current: {final_current} A")
print("\n✅ Advanced control session complete. Motor is stopped.")